In [3]:
# to do , I am using 2 different computational graphs, so need 2 different sessions
#[ done ] 

# use rnn to condense features into a single feature vector
#  [ pending ]


In [4]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from scipy.io import wavfile
import six
#import tensorflow as tf

import vggish_input
import vggish_params
import vggish_postprocess
import vggish_slim

/home/saurabh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
num_classes=4
data_base_directory='data/datav2/'
alphabet_dict={0:'A' , 1:'B' , 2:'C' , 3:'D' , 4:'E', 5:'F' , 6:'G' ,7 :'H' , 8:'I', 9:'J'}


In [6]:
def process_function(input):
  
  

  input=data_base_directory + str(input ,'utf-8')
  #print(str(input,'utf-8'))

 

  for i in range(num_classes):
     if alphabet_dict[i] in input:
         y=int(i)

  #if 'A' in str(input,'utf-8'):
   #     y=0
 


  
    
  return  input,y


In [7]:
logfilepath='data/dataset.txt'
dataset = tf.data.TextLineDataset(logfilepath).shuffle(buffer_size=200) #.map(map_function)

dataset = dataset.map(
    lambda filename: tuple(tf.py_func(
        process_function, [filename],  [tf.string,tf.int64])))

dataset=dataset.batch(3)


# In[25]:


iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()


# In[28]:

with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(200):
      value = sess.run(next_element[0])
      print(value)


[b'data/datav2/20_48_00018A.wav' b'data/datav2/20_50_46052A.wav'
 b'data/datav2/20_50_46120A.wav']
[b'data/datav2/20_50_46093A.wav' b'data/datav2/20_50_46098A.wav'
 b'data/datav2/20_50_46054A.wav']
[b'data/datav2/20_48_00006A.wav' b'data/datav2/20_48_00024A.wav'
 b'data/datav2/20_50_46032A.wav']
[b'data/datav2/20_50_46056A.wav' b'data/datav2/20_54_21030B.wav'
 b'data/datav2/20_50_46105A.wav']
[b'data/datav2/20_50_46116A.wav' b'data/datav2/20_50_46060A.wav'
 b'data/datav2/20_54_21039B.wav']
[b'data/datav2/20_54_21050B.wav' b'data/datav2/20_54_21036B.wav'
 b'data/datav2/20_54_21001B.wav']
[b'data/datav2/20_50_46085A.wav' b'data/datav2/20_48_00012A.wav'
 b'data/datav2/20_50_46040A.wav']
[b'data/datav2/20_50_46096A.wav' b'data/datav2/20_50_46084A.wav'
 b'data/datav2/20_50_46066A.wav']
[b'data/datav2/20_54_21012B.wav' b'data/datav2/20_50_46051A.wav'
 b'data/datav2/20_56_39006B.wav']
[b'data/datav2/20_50_46068A.wav' b'data/datav2/20_48_00005A.wav'
 b'data/datav2/20_50_46043A.wav']
[b'data/da

UnknownError: UnboundLocalError: local variable 'y' referenced before assignment
	 [[Node: PyFunc = PyFunc[Tin=[DT_STRING], Tout=[DT_STRING, DT_INT64], token="pyfunc_0"](arg0)]]
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[<unknown>, <unknown>], output_types=[DT_STRING, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Iterator)]]

In [9]:
# Parameters
learning_rate = 0.01
num_steps = 500
batch_size = 2
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 128 # MNIST data input (img shape: 28*28)
#num_classes = 5 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.int32, [None])


In [10]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [11]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 =  tf.nn.relu (tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    # Hidden fully connected layer with 256 neurons
    layer_2 =  tf.nn.relu(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [12]:
# Construct model
Logits = neural_net(X)

Logits = tf.Print(Logits, [Logits,tf.shape(Logits)], message="This is a: ")

# Define loss and optimizer
#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
 #   logits=logits, labels=Y))
prediction = tf.nn.softmax(Logits)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=Logits))
train_op = optimizer.minimize(cost)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(Logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))



# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [13]:
g_1 = tf.Graph()
with g_1.as_default():
  

                
                
  

  # Sessions created in this scope will run operations from `g_1`.
                sess1 = tf.Session()
        
                vggish_slim.define_vggish_slim(training=False)
                vggish_slim.load_vggish_slim_checkpoint(sess1, 'vggish_model.ckpt')
                features_tensor = sess1.graph.get_tensor_by_name(
                vggish_params.INPUT_TENSOR_NAME)
                embedding_tensor = sess1.graph.get_tensor_by_name(
                vggish_params.OUTPUT_TENSOR_NAME)



INFO:tensorflow:Restoring parameters from vggish_model.ckpt


In [18]:
# Start training
with tf.Session() as sess:
                sess.run(init)

   
                sess.run(iterator.initializer)

               
                
                examples_batch = vggish_input.wavfile_to_examples('data/vinay.wav')
                print(examples_batch.shape)

                # Prepare a postprocessor to munge the model embeddings.
                pproc = vggish_postprocess.Postprocessor('vggish_pca_params.npz')
                
               
    # Run inference and postprocessing.
                [embedding_batch] = sess1.run([embedding_tensor],
                                 feed_dict={features_tensor: examples_batch})
                print(embedding_batch.shape)
                postprocessed_batch = pproc.postprocess(embedding_batch)
                print(postprocessed_batch.shape)
    
    

              # train single example
                _ , c,l = sess.run([train_op, cost, Logits], feed_dict={X: postprocessed_batch,Y: Next_element[1] })
            
                 
                print(l.shape)
                print(c)
                #print(pre)
      #          print(p)
               
               
              
          
   
        
                
    # Save model weights to disk
   # save_path = saver.save(sess, model_path)
    #print("Model saved in file: %s" % save_path)   

(2, 96, 64)
(2, 128)
(2, 128)


InvalidArgumentError: logits and labels must have the same first dimension, got logits shape [2,4] and labels shape [1]
	 [[Node: SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Print, _arg_Placeholder_1_0_1)]]

Caused by op 'SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits', defined at:
  File "/home/saurabh/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/saurabh/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/saurabh/anaconda3/lib/python3.

In [14]:
# Start training
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(num_steps):
        avg_cost = 0.
        
        
        sess.run(iterator.initializer)
        while True:
            try:
                Next_element=sess.run(next_element)
                input_wav_file_name=str(Next_element[0][0],'utf-8')
                print(input_wav_file_name)
                
                examples_batch = vggish_input.wavfile_to_examples(input_wav_file_name)
                print(examples_batch.shape)

                # Prepare a postprocessor to munge the model embeddings.
                pproc = vggish_postprocess.Postprocessor('vggish_pca_params.npz')
                
               
    # Run inference and postprocessing.
                [embedding_batch] = sess1.run([embedding_tensor],
                                 feed_dict={features_tensor: examples_batch})
                print(embedding_batch.shape)
                postprocessed_batch = pproc.postprocess(embedding_batch)
                print(postprocessed_batch.shape)
    
    

              # train single example
                _ , c,l = sess.run([train_op, cost, Logits], feed_dict={X: postprocessed_batch,Y: Next_element[1] })
            
                 
                print(l.shape)
                print(c)
                #print(pre)
      #          print(p)
               
               
              
            except tf.errors.OutOfRangeError:
              break
   
        
                
    # Save model weights to disk
   # save_path = saver.save(sess, model_path)
    #print("Model saved in file: %s" % save_path)   

data/datav2/20_50_46060A.wav
(3, 96, 64)
(3, 128)
(3, 128)
(3, 4)
316399.66
data/datav2/20_50_46047A.wav
(3, 96, 64)
(3, 128)
(3, 128)
(3, 4)
96877.18
data/datav2/20_54_21009B.wav
(3, 96, 64)
(3, 128)
(3, 128)
(3, 4)
320554.22
data/datav2/20_50_46020A.wav
(3, 96, 64)
(3, 128)
(3, 128)
(3, 4)
0.0
data/datav2/20_50_46044A.wav
(3, 96, 64)
(3, 128)
(3, 128)
(3, 4)
101460.0
data/datav2/20_54_21038B.wav
(3, 96, 64)
(3, 128)
(3, 128)
(3, 4)
204539.73
data/datav2/20_50_46066A.wav
(3, 96, 64)
(3, 128)
(3, 128)
(3, 4)
0.0
data/datav2/20_50_46081A.wav
(3, 96, 64)


KeyboardInterrupt: 

In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))